In [1]:
cd("/Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/spinfoam-critical-pts")

In [2]:
using LinearAlgebra, Combinatorics
using Printf
using Dates

include("GeometryTypes.jl")          # defines GeometryDataset, GeometryCollection
#include("load_geometry_data.jl")        # low-level parsers FIRST
#include("GeometryDataLoader.jl")        # defines GeometryDataset
# --- include all modules ---
include("SimplexGeometry.jl")
include("SpinAlgebra.jl")
include("volume.jl")
include("TetraNormals.jl")
include("DihedralAngles.jl")
include("LorentzGroup.jl")
include("ThreeDTetra.jl")
include("Su2Su11FromBivector.jl")
include("XiFromSU.jl")
include("FaceNormals3D.jl")
include("KappaFromNormals.jl")
include("GeometryConsistency.jl")
include("GeometryPipeline.jl")
include("KappaOrientation.jl")
include("FourSimplexConnectivity.jl")
include("FaceXiMatching.jl")
include("FaceMatchingChecks.jl")
include("GaugeFixing.jl")
include("CriticalPoints.jl")


using .GeometryTypes: GeometryDataset, GeometryCollection
using .GeometryPipeline: run_geometry_pipeline
using .GeometryConsistency: check_sl2c_parallel_transport, check_so13_parallel_transport, check_closure_bivectors
using .KappaOrientation: fix_kappa_signs!
using .FourSimplexConnectivity: build_global_connectivity
using .FaceXiMatching: run_face_xi_matching
using .FaceMatchingChecks: check_all
using .GaugeFixingSU
using .CriticalPoints: compute_critical_data

In [3]:
bdypoints_all = [
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0],
        [0.0, 1.0, 1.0, 1.0],
        [0.5, 1.0, 1.0, 1.0]
    ],
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 1.0, 1.0, 1.0],
        [0.5, 1.0, 1.0, 1.0],
        [0.0, 1.0, 0.0, 1.0]
    ]
]

2-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 1.0], [0.0, 1.0, 1.0, 1.0], [0.5, 1.0, 1.0, 1.0]]
 [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.0, 1.0, 1.0, 1.0], [0.5, 1.0, 1.0, 1.0], [0.0, 1.0, 0.0, 1.0]]

In [4]:
geom = nothing
datasets = GeometryDataset[]
ns = length(bdypoints_all)

base_folder = "geom_output/new-data/"
mkpath(base_folder)

for s in 1:ns
    bdypoints = bdypoints_all[s]
    println("Boundary points for simplex $s:")

    ds = run_geometry_pipeline(bdypoints)  # GeometryDataset
    push!(datasets, ds)

end

Boundary points for simplex 1:
Boundary points for simplex 2:


In [5]:
geom = GeometryCollection(datasets);

In [6]:
four_simplices = [[1,2,3,4,5], [1,2,4,5,6]];

fix_kappa_signs!(four_simplices, geom);

build_global_connectivity(four_simplices, geom);

[1, 2]


In [7]:
conn = build_global_connectivity(four_simplices, geom)
push!(geom.connectivity, conn);

In [8]:
run_face_xi_matching(geom);

  SL(2,C) matrices updated:      ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓



In [9]:
check_all(geom)


   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 2.8389806009761813e-16).
✓ SL(2,C) parallel transport satisfied (max residual = 6.482575983860851e-16).
✓ Closure condition satisfied (max residual = 2.5133742693021536e-16).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------



In [10]:
run_su2_su11_gauge_fix(geom);

In [12]:
γ = 1
crit = CriticalPoints.compute_critical_data(geom; gamma = γ)

(gdataof = Vector{Matrix{ComplexF64}}[[[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im], [1.037954849302042 + 0.0im 0.27811916365044975 + 0.0im; 0.27811916365044975 - 0.0im 1.0379548493020423 - 0.0im], [0.45576803893928236 - 0.5406250962371617im -0.7045563426109881 - 0.06000300064686592im; 0.7045563426109882 - 0.06000300064686581im 0.45576803893928264 + 0.540625096237162im], [0.0 - 0.3826834323650896im 0.9238795325112867 - 0.0im; -0.923879532511287 + 0.0im 0.0 + 0.3826834323650897im], [0.0 - 1.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 1.0im]], [[-0.45576803893928236 + 0.540625096237162im 0.7045563426109881 + 0.060003000646866145im; -0.704556342610988 + 0.060003000646866034im -0.45576803893928236 - 0.5406250962371619im], [1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im], [1.037954849302042 + 0.0im 0.0 + 0.27811916365044975im; 0.0 - 0.27811916365044975im 1.0379548493020423 - 0.0im], [0.0 - 0.3826834323650896im 0.0 - 0.9238795325112867im; 0.0 - 0.923879532511287im 0.0 + 0.3826834323650897im],

In [ ]:
function build_xi_expressions(sgndet, tetareasign, tetn0, ns)

    xi_expr = Vector{Any}(undef, ns)
    for k in 1:ns
        xi_expr[k] = Vector{Any}(undef, 5)
        for i in 1:5
            xi_expr[k][i] = Vector{Any}(undef, 5)
        end
    end

    for k in 1:ns
        for i in 1:5
            for j in 1:5

                if i == j
                    xi_expr[k][i][j] = (0, 0)
                    continue
                end

                # symbolic names
                a = Symbol("zeta_$(k)_$(i)_$(j)_a")
                b = Symbol("zeta_$(k)_$(i)_$(j)_b")

                if sgndet[k][i] == 1
                    # ( sin(a) , cos(a)*exp(i b) )
                    xi_expr[k][i][j] = (
                        :(sin($a)),
                        :(cos($a) * exp(im*$b))
                    )

                elseif tetareasign[k][i][j] == 1
                    if tetn0[k][i][j] == 1
                        xi_expr[k][i][j] = (
                            :(cosh($a)),
                            :(exp(-im*$b) * sinh($a))
                        )
                    else
                        xi_expr[k][i][j] = (
                            :(sinh($a) * exp(im*$b)),
                            :(cosh($a))
                        )
                    end

                else
                    # null/timelike case (1 , exp(i b))
                    xi_expr[k][i][j] = (
                        1,
                        :(exp(im*$b))
                    )
                end
            end
        end
    end

    return xi_expr
end

function apply_shared_tets_to_xi!(xi_expr, sharedTetsPos)
    ns   = length(xi_expr)
    ntet = length(xi_expr[1])   # 5 in your data

    for pair in sharedTetsPos
        s1 = pair[1][1]
        t1 = pair[1][2]
        s2 = pair[2][1]
        t2 = pair[2][2]

        @assert 1 ≤ s1 ≤ ns && 1 ≤ s2 ≤ ns
        @assert 1 ≤ t1 ≤ ntet && 1 ≤ t2 ≤ ntet

        # row of ξ's for (s1,t1): this is the list over j
        row_src = xi_expr[s1][t1]  # Vector{Any} of length ntet

        # Delete[...] at position t1
        row_wo_self = Any[]
        for j in 1:ntet
            j == t1 && continue
            push!(row_wo_self, row_src[j])
        end

        # Insert {1,0} at position t2
        row_dst = Any[]
        for j in 1:ntet
            if j == t2
                push!(row_dst, (1, 0))   # matches {1,0} in MMA (no ζ here)
            elseif j < t2
                push!(row_dst, row_wo_self[j])
            else
                push!(row_dst, row_wo_self[j-1])
            end
        end

        xi_expr[s2][t2] = row_dst
    end

    return xi_expr
end

# helper: collect all symbols inside an Expr / Tuple / Array
function _collect_syms!(vars::Vector{Symbol}, x)
    if x isa Symbol
        # ignore standard builtins and constants
        if !(x in (:sin, :cos, :sinh, :cosh, :exp, :im, :*, :^, :/, :+, :-, :π, :E))
            push!(vars, x)
        end
    elseif x isa Expr
        for arg in x.args
            _collect_syms!(vars, arg)
        end
    elseif x isa Tuple || x isa AbstractArray
        for elem in x
            _collect_syms!(vars, elem)
        end
    end
end

"""
    build_zetavariables(xi_expr)

Given xi_expr[k][i][j] = (expr1, expr2) from `build_xi_expressions`,
return `zetavars[k][i][j]::Vector{Symbol}` listing the zeta symbols
for that (k,i,j), in a deterministic order.
"""
function build_zetavariables(xi_expr)
    ns   = length(xi_expr)
    ntet = length(xi_expr[1])

    zetavars = Vector{Vector{Vector{Vector{Symbol}}}}(undef, ns)

    for k in 1:ns
        zetavars[k] = Vector{Vector{Vector{Symbol}}}(undef, ntet)
        for i in 1:ntet
            zetavars[k][i] = Vector{Vector{Symbol}}(undef, ntet)
            for j in 1:ntet
                vars = Symbol[]
                x = xi_expr[k][i][j]

                # x is either (expr1, expr2) or (0,0)
                _collect_syms!(vars, x)

                # remove duplicates and sort to fix order (like MMA Sort)
                zetavars[k][i][j] = sort(unique(vars))
            end
        end
    end

    return zetavars
end

"""
    build_zetadata(xisoln, zetavars, tetareasign)

Return Dict{Symbol,Float64} giving the numerical value of each ζ–variable
at the critical point.

- If tetareasign[k][i][j] == 1:  both ζₐ, ζ_b get values from xisoln[k][i][j]
- Else: only ζ_b gets value xisoln[k][i][j][1]
"""
function build_zetadata(xisoln, zetavars, tetareasign)
    ns   = length(xisoln)
    ntet = length(xisoln[1])

    zeta_data = Dict{Symbol,Float64}()

    for k in 1:ns
        for i in 1:ntet
            for j in 1:ntet
                # skip diagonal if you like (i == j)
                if i == j
                    continue
                end

                vars = zetavars[k][i][j]
                isempty(vars) && continue

                if tetareasign[k][i][j] == 1
                    vals = xisoln[k][i][j]             # [θ, φ]
                else
                    vals = [xisoln[k][i][j][1]]        # only θ (like MMA)
                end

                @assert length(vars) == length(vals) "Mismatch vars vs vals at (k,i,j) = ($k,$i,$j)"

                for (sym, val) in zip(vars, vals)
                    zeta_data[sym] = val
                end
            end
        end
    end

    return zeta_data
end

"""
    compute_gspecialpos(gdataof, GaugeTet; tol=1e-12)

Return the list of (k,i) where gdataof[k][i][1,1] ≈ 0
and (k,i) is *not* in GaugeTet.
GaugeTet is taken from geom.connectivity[1]["GaugeTet"].
"""
function compute_gspecialpos(gdataof, GaugeTet; tol=1e-12)
    ns   = length(gdataof)
    ntet = length(gdataof[1])

    # Convert GaugeTet to a Set of Vectors {k,i}
    gauge_set = Set{Vector{Int}}()
    for p in GaugeTet
        push!(gauge_set, [p[1], p[2]])
    end

    gspecialpos = Vector{Vector{Int}}()   # final output

    for k in 1:ns
        for i in 1:ntet

            key = [k, i]     # always a Vector{Int}

            # skip gauge-fixed tetrahedra
            if key ∈ gauge_set
                continue
            end

            # check g[1,1] ≈ 0
            if abs(gdataof[k][i][1,1]) < tol
                push!(gspecialpos, key)
            end
        end
    end

    return gspecialpos
end

"""
    compute_zspecialpos(zdataf, kappa; tol=1e-12)

Return list of [k,i,j] where
    kappa[k][i][j] == 1, i ≠ j,
    and the first component zdataf[k][i][j][1] is ≈ 1 (modulus).
Matches the Mathematica definition of zspecialPos.
"""
function compute_zspecialpos(zdataf, kappa; tol=1e-12)
    ns   = length(zdataf)
    ntet = length(zdataf[1])

    out = Vector{Vector{Int}}()

    for k in 1:ns
        for i in 1:ntet
            for j in 1:ntet

                # Mathematica: only when kappa==1 and i != j
                if kappa[k][i][j] == 1 && i != j

                    z1 = zdataf[k][i][j][1]

                    # Mathematica condition:
                    # If Chop[N[z1]] == 1 -> skip
                    # else push
                    if !(abs(z1 - 1) < tol)
                        push!(out, [k, i, j])
                    end
                end
            end
        end
    end

    return out
end

"""
    build_gvariablesall(ns, ntet, GaugeTet, gspecialpos, GaugeFixUpperTriangle)

Return gvariablesall[k][i] = 2×2 matrix of Expr,
with branches matching the Mathematica definition.
"""
# det([[a b]; [c d]]) == 1  =>  d = (1 + b*c)/a
solve_det_for_d(a, b, c) = :((1 + $b*$c) / $a)

# membership for 2-int positions stored as Vector{Vector{Int}}
@inline function in_pos2(pos::Vector{Int}, lst::Vector{Vector{Int}})
    @inbounds for q in lst
        if q[1] == pos[1] && q[2] == pos[2]
            return true
        end
    end
    return false
end

function build_gvariablesall(ns::Int, ntet::Int,
                             GaugeTet::Vector{Vector{Int}},
                             gspecialpos::Vector{Vector{Int}},
                             GaugeFixUpperTriangle::Vector{Vector{Int}})

    gvars = Vector{Vector{Matrix{Any}}}(undef, ns)

    for k in 1:ns
        gvars[k] = Vector{Matrix{Any}}(undef, ntet)

        for i in 1:ntet
            pos = [k, i]

            # ------------------------------------------------------
            # 1) GaugeTet: full 8 real params (no det solving)
            # ------------------------------------------------------
            if in_pos2(pos, GaugeTet)
                Ga  = Symbol("Ga$(k)_$(i)")
                Gac = Symbol("Gac$(k)_$(i)")
                Gb  = Symbol("Gb$(k)_$(i)")
                Gbc = Symbol("Gbc$(k)_$(i)")
                Gc  = Symbol("Gc$(k)_$(i)")
                Gcc = Symbol("Gcc$(k)_$(i)")
                Gd  = Symbol("Gd$(k)_$(i)")
                Gdc = Symbol("Gdc$(k)_$(i)")

                gvars[k][i] = Any[
                    :($Ga + im*$Gac)   :($Gb + im*$Gbc);
                    :($Gc + im*$Gcc)   :($Gd + im*$Gdc)
                ]

            # ------------------------------------------------------
            # 2) gspecialpos: det=1 solved for d
            # ------------------------------------------------------
            elseif in_pos2(pos, gspecialpos)
                ga  = Symbol("ga$(k)_$(i)")
                gac = Symbol("gac$(k)_$(i)")
                gb  = Symbol("gb$(k)_$(i)")
                gbc = Symbol("gbc$(k)_$(i)")
                gc  = Symbol("gc$(k)_$(i)")
                gcc = Symbol("gcc$(k)_$(i)")

                a = :(1 + $ga + im*$gac)
                b = :($gb + im*$gbc)
                c = :($gc + im*$gcc)
                d = solve_det_for_d(a, b, c)

                gvars[k][i] = Any[a b; c d]

            # ------------------------------------------------------
            # 3) GaugeFixUpperTriangle: triangular + det=1 solve
            # ------------------------------------------------------
            elseif in_pos2(pos, GaugeFixUpperTriangle)
                ga  = Symbol("ga$(k)_$(i)")
                gc  = Symbol("gc$(k)_$(i)")
                gcc = Symbol("gcc$(k)_$(i)")

                a = :(1 + $ga)
                b = 0
                c = :($gc + im*$gcc)
                d = solve_det_for_d(a, b, c)

                gvars[k][i] = Any[a 0; c d]

            # ------------------------------------------------------
            # 4) generic: det=1 solved for d
            # ------------------------------------------------------
            else
                ga  = Symbol("ga$(k)_$(i)")
                gac = Symbol("gac$(k)_$(i)")
                gb  = Symbol("gb$(k)_$(i)")
                gbc = Symbol("gbc$(k)_$(i)")
                gc  = Symbol("gc$(k)_$(i)")
                gcc = Symbol("gcc$(k)_$(i)")

                a = :(1 + $ga + im*$gac)
                b = :($gb + im*$gbc)
                c = :($gc + im*$gcc)
                d = solve_det_for_d(a, b, c)

                gvars[k][i] = Any[a b; c d]
            end
        end
    end

    return gvars
end

function build_zvariablesall(ns, ntet, kappa, zspecialPos)
    special_set = Set{NTuple{3,Int}}((p[1],p[2],p[3]) for p in zspecialPos)

    zvars = Vector{Vector{Vector{Vector{Any}}}}(undef, ns)

    for k in 1:ns
        zvars[k] = Vector{Vector{Vector{Any}}}(undef, ntet)
        for i in 1:ntet
            zvars[k][i] = Vector{Vector{Any}}(undef, ntet)

            for j in 1:ntet
                if kappa[k][i][j] == 1 && i != j
                    z  = Symbol("z$(k)_$(i)$(j)")
                    zc = Symbol("zc$(k)_$(i)$(j)")
                    key = (k,i,j)

                    if key in special_set
                        zvars[k][i][j] = Any[:($z + im*$zc), 1]
                    else
                        zvars[k][i][j] = Any[1, :($z + im*$zc)]
                    end
                else
                    zvars[k][i][j] = Any[0,0]
                end
            end
        end
    end

    return zvars
end

function collect_symbols_any(x)
    vars = Symbol[]
    _collect_syms!(vars, x)
    return unique(vars)
end

collect_symbols_any (generic function with 1 method)

In [54]:
ntet = length(geom.simplex[1].bdyxi)
kappa = [geom.simplex[s].kappa for s in 1:ns]

xi_final     = [geom.simplex[s].bdyxi       for s in 1:ns]
sgndet       = [geom.simplex[s].sgndet      for s in 1:ns]
tetareasign  = [geom.simplex[s].tetareasign for s in 1:ns]
tetn0        = [geom.simplex[s].tetn0sign   for s in 1:ns]

# 1) symbolic ξ(zeta)
xi_expr = build_xi_expressions(sgndet, tetareasign, tetn0, ns)

# impose shared-tet identification like in Mathematica
sharedTetsPos = geom.connectivity[1]["sharedTetsPos"]
apply_shared_tets_to_xi!(xi_expr, sharedTetsPos)

# 2) ζ variables
zetavars  = build_zetavariables(xi_expr)

# 3) critical data
crit     = compute_critical_data(geom;gamma = 1)
xisoln   = crit.xisoln
gdataof  = crit.gdataof
zdataf   = crit.zdataf
jdataf   = crit.jdataf

# 4) ζ numeric solution
zeta_data = build_zetadata(xisoln, zetavars, tetareasign)

GaugeTet             = geom.connectivity[1]["GaugeTet"]
GaugeFixUpperTriangle = geom.connectivity[1]["GaugeFixUpperTriangle"]

Tuple{Int64, Int64}[]

In [55]:
gspecialpos = compute_gspecialpos(gdataof, GaugeTet)

Vector{Int64}[]

In [56]:
zspecialPos = compute_zspecialpos(zdataf, kappa)

2-element Vector{Vector{Int64}}:
 [1, 5, 1]
 [2, 5, 2]

In [57]:
gvariablesall = build_gvariablesall(ns, ntet, GaugeTet, gspecialpos, GaugeFixUpperTriangle)

2-element Vector{Vector{Matrix{Any}}}:
 [[:(Ga1_1 + im * Gac1_1) :(Gb1_1 + im * Gbc1_1); :(Gc1_1 + im * Gcc1_1) :(Gd1_1 + im * Gdc1_1)], [:(1 + ga1_2 + im * gac1_2) :(gb1_2 + im * gbc1_2); :(gc1_2 + im * gcc1_2) :((1 + (gb1_2 + im * gbc1_2) * (gc1_2 + im * gcc1_2)) / (1 + ga1_2 + im * gac1_2))], [:(1 + ga1_3 + im * gac1_3) :(gb1_3 + im * gbc1_3); :(gc1_3 + im * gcc1_3) :((1 + (gb1_3 + im * gbc1_3) * (gc1_3 + im * gcc1_3)) / (1 + ga1_3 + im * gac1_3))], [:(1 + ga1_4 + im * gac1_4) :(gb1_4 + im * gbc1_4); :(gc1_4 + im * gcc1_4) :((1 + (gb1_4 + im * gbc1_4) * (gc1_4 + im * gcc1_4)) / (1 + ga1_4 + im * gac1_4))], [:(1 + ga1_5 + im * gac1_5) :(gb1_5 + im * gbc1_5); :(gc1_5 + im * gcc1_5) :((1 + (gb1_5 + im * gbc1_5) * (gc1_5 + im * gcc1_5)) / (1 + ga1_5 + im * gac1_5))]]
 [[:(1 + ga2_1 + im * gac2_1) :(gb2_1 + im * gbc2_1); :(gc2_1 + im * gcc2_1) :((1 + (gb2_1 + im * gbc2_1) * (gc2_1 + im * gcc2_1)) / (1 + ga2_1 + im * gac2_1))], [:(Ga2_2 + im * Gac2_2) :(Gb2_2 + im * Gbc2_2); :(Gc2_2 + im 

In [58]:
zvariablesall = build_zvariablesall(ns, ntet, kappa, zspecialPos)

2-element Vector{Vector{Vector{Vector{Any}}}}:
 [[[0, 0], [1, :(z1_12 + im * zc1_12)], [0, 0], [1, :(z1_14 + im * zc1_14)], [0, 0]], [[0, 0], [0, 0], [1, :(z1_23 + im * zc1_23)], [0, 0], [1, :(z1_25 + im * zc1_25)]], [[1, :(z1_31 + im * zc1_31)], [0, 0], [0, 0], [1, :(z1_34 + im * zc1_34)], [0, 0]], [[0, 0], [1, :(z1_42 + im * zc1_42)], [0, 0], [0, 0], [1, :(z1_45 + im * zc1_45)]], [[:(z1_51 + im * zc1_51), 1], [0, 0], [1, :(z1_53 + im * zc1_53)], [0, 0], [0, 0]]]
 [[[0, 0], [0, 0], [1, :(z2_13 + im * zc2_13)], [0, 0], [1, :(z2_15 + im * zc2_15)]], [[1, :(z2_21 + im * zc2_21)], [0, 0], [0, 0], [1, :(z2_24 + im * zc2_24)], [0, 0]], [[0, 0], [1, :(z2_32 + im * zc2_32)], [0, 0], [0, 0], [1, :(z2_35 + im * zc2_35)]], [[1, :(z2_41 + im * zc2_41)], [0, 0], [1, :(z2_43 + im * zc2_43)], [0, 0], [0, 0]], [[0, 0], [:(z2_52 + im * zc2_52), 1], [0, 0], [1, :(z2_54 + im * zc2_54)], [0, 0]]]

In [59]:
# zetavaria = collect_symbols_any(zetavars)
# gvaria  = collect_symbols_any(gvariablesall)
# zvaria  = collect_symbols_any(zvariablesall)